In [1]:
# Import necessary libraries
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress INFO and WARNING messages
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks
from sklearn.model_selection import KFold
from itertools import product
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier

# Import additional libraries for Random Forest and SVM
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import (
    accuracy_score, confusion_matrix, f1_score, roc_auc_score, roc_curve
)
import random 

2025-07-16 16:04:03.836822: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-16 16:04:04.526916: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-16 16:04:04.908495: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
import sys
import types

for name, val in sys.modules.items():
    if isinstance(val, types.ModuleType) and hasattr(val, '__version__'):
        try:
            print(f"{name}=={val.__version__}")
        except Exception:
            pass

re==2.2.1
ipaddress==1.0
ipykernel._version==6.29.5
json==2.0.9
jupyter_client._version==8.6.2
platform==1.0.8
zmq.sugar.version==26.1.0
zmq.sugar==26.1.0
zmq==26.1.0
logging==0.5.1.2
traitlets._version==5.14.3
traitlets==5.14.3
jupyter_core.version==5.7.2
jupyter_core==5.7.2
zlib==1.0
_curses==b'2.2'
socketserver==0.4
argparse==1.1
dateutil._version==2.9.0
dateutil==2.9.0
six==1.16.0
_decimal==1.70
decimal==1.70
platformdirs.version==4.2.2
platformdirs==4.2.2
_csv==1.0
csv==1.0
jupyter_client==8.6.2
ipykernel==6.29.5
IPython.core.release==8.26.0
executing.version==2.0.1
executing==2.0.1
pure_eval.version==0.2.3
pure_eval==0.2.3
stack_data.version==0.6.2
stack_data==0.6.2
pygments==2.18.0
pickleshare==0.7.5
decorator==5.1.1
wcwidth==0.2.13
prompt_toolkit==3.0.47
parso==0.8.4
jedi==0.19.1
urllib.request==3.12
IPython==8.26.0
comm==0.2.2
psutil==6.0.0
packaging==24.1
_ctypes==1.1.0
ctypes==1.1.0
debugpy.public_api==1.8.5
debugpy==1.8.5
xmlrpc.client==3.12
http.server==0.6
_pydevd_frame_e

In [8]:
# 1. Set a seed for numpy (used in arrays, random number generation)
np.random.seed(123)

# 2. Set a seed for Python's built-in random module
random.seed(123)

# 3. Set a seed for TensorFlow/Keras (if using neural networks)
tf.random.set_seed(123)

In [3]:
# For TensorFlow, you can also disable GPU and other non-deterministic behaviors (optional, based on the use case)
tf.config.experimental.enable_op_determinism()

In [4]:
data = pd.read_csv('20240923_TrainingDataSet_Clean.csv')
feature_columns = data.columns.drop(['Unnamed: 0', 'target', 'CADD_METSIM'])

# Display the first few rows of the DataFrame
data['target'] = data['target'].replace(-1, 0)
data = data.replace([np.inf, -np.inf], np.nan)

In [254]:
# Implement 5-fold cross-validation
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=123)

# Define reduced hyperparameter grid for neural network
param_grid = {
    'units1': [32, 64],
    'units2': [16, 32],
    'dropout_rate': [0.2],     # Fixed dropout rate
    'batch_size': [32],        # Fixed batch size
    'epochs': [50],            # Fixed number of epochs
    'lr': [0.001, 0.01]
}

hyperparameter_list = list(product(
    param_grid['units1'],
    param_grid['units2'],
    param_grid['dropout_rate'],
    param_grid['batch_size'],
    param_grid['epochs'],
    param_grid['lr']
))

# Initialize results list for neural network
nn_results_list = []

# Initialize results lists for Random Forest and SVM
rf_metrics_list = []
svm_metrics_list = []
# Initialize the list to store Gradient Boosting metrics
gb_metrics_list = []

In [255]:
# Loop over folds
fold = 1
for train_index, val_index in kf.split(data):
    print(f"\nProcessing Fold {fold}")
    
    # Split data into training and validation sets
    training_data = data.iloc[train_index].reset_index(drop=True)
    validation_data = data.iloc[val_index].reset_index(drop=True)
    
    # Impute missing values (median imputation)
    # Calculate median of each feature in training data
    medians = training_data[feature_columns].median()
    
    # Fill missing values in training data
    training_data_imputed = training_data.copy()
    training_data_imputed[feature_columns] = training_data_imputed[feature_columns].fillna(medians)
    
    # Fill missing values in validation data using training medians
    validation_data_imputed = validation_data.copy()
    validation_data_imputed[feature_columns] = validation_data_imputed[feature_columns].fillna(medians)
    
    # Separate features and target
    x_train = training_data_imputed[feature_columns].values
    y_train = training_data_imputed['target'].values
    x_val = validation_data_imputed[feature_columns].values
    y_val = validation_data_imputed['target'].values
    
    # Scale features using training data statistics
    feature_means = x_train.mean(axis=0)
    feature_stds = x_train.std(axis=0)
    x_train_scaled = (x_train - feature_means) / feature_stds
    x_val_scaled = (x_val - feature_means) / feature_stds  # Use training data stats

    ### Neural Network ###
    # Loop over hyperparameter grid
    for params in hyperparameter_list:
        units1, units2, dropout_rate, batch_size, epochs, lr = params
        
        print(f"  Neural Network Hyperparameters: units1 = {units1}, units2 = {units2}, dropout_rate = {dropout_rate}, "
              f"batch_size = {batch_size}, epochs = {epochs}, lr = {lr}")
        
        # Build model
        model = keras.Sequential([
            keras.Input(shape=(x_train_scaled.shape[1],)),
            layers.Dense(units1, activation='relu'),
            layers.Dropout(dropout_rate),
            layers.Dense(units2, activation='relu'),
            layers.Dropout(dropout_rate),
            layers.Dense(1, activation='sigmoid')
        ])
        
        # Compile model
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=lr),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        # Include early stopping
        early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        
        # Train model
        history = model.fit(
            x_train_scaled, y_train,
            epochs=epochs,
            batch_size=batch_size,
            validation_data=(x_val_scaled, y_val),
            verbose=0,  # Set to 1 to see training progress
            callbacks=[early_stopping]
        )
        
        # Evaluate model on validation data
        evaluation = model.evaluate(x_val_scaled, y_val, verbose=0)
        
        # Make predictions
        y_pred_prob_nn = model.predict(x_val_scaled).flatten()
        y_pred_nn = (y_pred_prob_nn >= 0.5).astype(int)
        
        # Compute evaluation metrics
        nn_f1 = f1_score(y_val, y_pred_nn)
        nn_auc = roc_auc_score(y_val, y_pred_prob_nn)
        tn, fp, fn, tp = confusion_matrix(y_val, y_pred_nn).ravel()
        nn_tpr = tp / (tp + fn)  # True Positive Rate
        nn_tnr = tn / (tn + fp)  # True Negative Rate
        nn_fpr = fp / (fp + tn)  # False Positive Rate
        nn_fnr = fn / (fn + tp)  # False Negative Rate
        
        # Record results
        nn_results_list.append({
            'fold': fold,
            'units1': units1,
            'units2': units2,
            'dropout_rate': dropout_rate,
            'batch_size': batch_size,
            'epochs': epochs,
            'lr': lr,
            'trained_epochs': len(history.history['loss']),
            'val_loss': evaluation[0],
            'val_accuracy': evaluation[1],
            'f1_score': nn_f1,
            'auc': nn_auc,
            'tp': tp,
            'tn': tn,
            'fp': fp,
            'fn': fn,
            'tpr': nn_tpr,
            'tnr': nn_tnr,
            'fpr': nn_fpr,
            'fnr': nn_fnr
        })
    
    ### Random Forest ###
    # Instantiate Random Forest classifier with default hyperparameters
    rf_model = RandomForestClassifier(random_state=123, min_samples_split=5, max_depth=10, n_estimators=200)
    
    # Train Random Forest model
    rf_model.fit(x_train, y_train)
    
    # Predict on validation data
    rf_pred = rf_model.predict(x_val)
    rf_pred_prob = rf_model.predict_proba(x_val)[:, 1]
    
    # Compute evaluation metrics
    rf_accuracy = accuracy_score(y_val, rf_pred)
    rf_f1 = f1_score(y_val, rf_pred)
    rf_auc = roc_auc_score(y_val, rf_pred_prob)
   
    tn, fp, fn, tp = confusion_matrix(y_val, rf_pred).ravel()
    rf_tpr = tp / (tp + fn)
    rf_tnr = tn / (tn + fp)
    rf_fpr = fp / (fp + tn)
    rf_fnr = fn / (fn + tp)
    rf_fpr_roc, rf_tpr_roc, thresholds = roc_curve(y_val, rf_pred_prob)
    
    # Find the index of the FPR desired
    desired_fpr = 0.05
    closest_idx = np.argmin(np.abs(rf_fpr_roc - desired_fpr))
    optimal_threshold_FPR = thresholds[closest_idx]

    # Record results
    rf_metrics_list.append({
        'fold': fold,
        'accuracy': rf_accuracy,
        'f1_score': rf_f1,
        'auc': rf_auc,
        'tp': tp,
        'tn': tn,
        'fp': fp,
        'fn': fn,
        'tpr': rf_tpr,
        'tnr': rf_tnr,
        'fpr': rf_fpr,
        'fnr': rf_fnr,
        'FPR_threshold': optimal_threshold_FPR
    })

    output_random_forest = pd.DataFrame({
    'rf_fpr': rf_fpr_roc,
    'rf_tpr': rf_tpr_roc,
    'thresh': thresholds
    })

    output_random_forest.to_csv(f"/output_random_forest_for_roc_{fold}.csv", index=False)
    print(f"  Random Forest Accuracy: {rf_accuracy:.4f}, F1 Score: {rf_f1:.4f}, AUC: {rf_auc:.4f}")

    ### Gradient
    # Instantiate Gradient Boosting classifier with default hyperparameters
    gb_model = GradientBoostingClassifier(random_state=123)

    # Train Gradient Boosting model
    gb_model.fit(x_train, y_train)

    # Predict on validation data
    gb_pred = gb_model.predict(x_val)
    gb_pred_prob = gb_model.predict_proba(x_val)[:, 1]
    
    # Compute evaluation metrics
    gb_accuracy = accuracy_score(y_val, gb_pred)
    gb_f1 = f1_score(y_val, gb_pred)
    gb_auc = roc_auc_score(y_val, gb_pred_prob)
    tn, fp, fn, tp = confusion_matrix(y_val, gb_pred).ravel()
    gb_tpr = tp / (tp + fn)  # True Positive Rate (Recall)
    gb_tnr = tn / (tn + fp)  # True Negative Rate
    gb_fpr = fp / (fp + tn)  # False Positive Rate
    gb_fnr = fn / (fn + tp)  # False Negative Rate
    
    # Record results
    gb_metrics_list.append({
        'fold': fold,
        'accuracy': gb_accuracy,
        'f1_score': gb_f1,
        'auc': gb_auc,
        'tp': tp,
        'tn': tn,
        'fp': fp,
        'fn': fn,
        'tpr': gb_tpr,
        'tnr': gb_tnr,
        'fpr': gb_fpr,
        'fnr': gb_fnr
    })

    # Print results
    print(f"  Gradient Boosting Accuracy: {gb_accuracy:.4f}, F1 Score: {gb_f1:.4f}, AUC: {gb_auc:.4f}")

    ### Support Vector Machine ###
    # Instantiate SVM classifier with default hyperparameters
    svm_model = SVC(kernel='rbf', probability=True, random_state=123)
    
    # Train SVM model
    svm_model.fit(x_train_scaled, y_train)
    
    # Predict on validation data
    svm_pred = svm_model.predict(x_val_scaled)
    svm_pred_prob = svm_model.predict_proba(x_val_scaled)[:, 1]
    
    # Compute evaluation metrics
    svm_accuracy = accuracy_score(y_val, svm_pred)
    svm_f1 = f1_score(y_val, svm_pred)
    svm_auc = roc_auc_score(y_val, svm_pred_prob)
    tn, fp, fn, tp = confusion_matrix(y_val, svm_pred).ravel()
    svm_tpr = tp / (tp + fn)
    svm_tnr = tn / (tn + fp)
    svm_fpr = fp / (fp + tn)
    svm_fnr = fn / (fn + tp)

    # Record results
    svm_metrics_list.append({
        'fold': fold,
        'accuracy': svm_accuracy,
        'f1_score': svm_f1,
        'auc': svm_auc,
        'tp': tp,
        'tn': tn,
        'fp': fp,
        'fn': fn,
        'tpr': svm_tpr,
        'tnr': svm_tnr,
        'fpr': svm_fpr,
        'fnr': svm_fnr
    })
    
    print(f"  SVM Accuracy: {svm_accuracy:.4f}, F1 Score: {svm_f1:.4f}, AUC: {svm_auc:.4f}")
    
    fold += 1
    


Processing Fold 1
  Neural Network Hyperparameters: units1 = 32, units2 = 16, dropout_rate = 0.2, batch_size = 32, epochs = 50, lr = 0.001
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
  Neural Network Hyperparameters: units1 = 32, units2 = 16, dropout_rate = 0.2, batch_size = 32, epochs = 50, lr = 0.01
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
  Neural Network Hyperparameters: units1 = 32, units2 = 32, dropout_rate = 0.2, batch_size = 32, epochs = 50, lr = 0.001
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
  Neural Network Hyperparameters: units1 = 32, units2 = 32, dropout_rate = 0.2, batch_size = 32, epochs = 50, lr = 0.01
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
  Neural Network Hyperparameters: units1 = 64, units2 = 16, dropout_rate = 0.2, batch_size = 32, epochs = 50, lr = 0.001
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
  Neural Network Hyperparameters: units1 = 64, units2 = 16, dropout_rate = 0.2, batch_size = 32, epochs = 50, lr = 0.01
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
  Neural Network Hyperparamete

In [256]:
# After the loops, create DataFrames for the results
nn_results = pd.DataFrame(nn_results_list)
rf_results = pd.DataFrame(rf_metrics_list)
svm_results = pd.DataFrame(svm_metrics_list)
gb_results = pd.DataFrame(gb_metrics_list)

# Aggregate neural network results over folds
mean_nn_results = nn_results.groupby(['units1', 'units2', 'dropout_rate', 'batch_size', 'epochs', 'lr'], as_index=False).agg({
    'val_loss': 'mean',
    'val_accuracy': 'mean',
    'f1_score': 'mean',
    'auc': 'mean',
    'tpr': 'mean',
    'tnr': 'mean',
    'fpr': 'mean',
    'fnr': 'mean'
}).sort_values(by='val_accuracy', ascending=False)

In [257]:
# output all the files
nn_results.to_csv('nn_results.csv', index=False)  # Set index=False to avoid writing row numbers
rf_results.to_csv('rf_results.csv', index=False)  # Set index=False to avoid writing row numbers
svm_results.to_csv('svm_results.csv', index=False)  # Set index=False to avoid writing row numbers
mean_nn_results.to_csv('mean_nn_results.csv', index=False)  # Set index=False to avoid writing row numbers

In [272]:
gb_results.to_csv('mean_gb_results.csv', index=False)  # Set index=False to avoid writing row numbers


In [258]:
print(rf_results)
# print(svm_results)

   fold  accuracy  f1_score       auc  tp   tn  fp  fn       tpr       tnr  \
0     1  0.657895  0.599119  0.700650  68  107  28  63  0.519084  0.792593   
1     2  0.691729  0.663934  0.752000  81  103  22  60  0.574468  0.824000   
2     3  0.706767  0.669492  0.751077  79  109  31  47  0.626984  0.778571   
3     4  0.706767  0.677686  0.762040  82  106  39  39  0.677686  0.731034   
4     5  0.654135  0.646154  0.711016  84   90  30  62  0.575342  0.750000   

        fpr       fnr  FPR_threshold  
0  0.207407  0.480916       0.706707  
1  0.176000  0.425532       0.683409  
2  0.221429  0.373016       0.701180  
3  0.268966  0.322314       0.702879  
4  0.250000  0.424658       0.694702  


In [259]:
# Show the best hyperparameters
print("\nNeural Network Mean Validation Metrics:")
print(mean_nn_results)

# Calculate mean metrics for Random Forest and SVM
mean_rf_results = rf_results.mean()
mean_svm_results = svm_results.mean()
mean_gb_results = gb_results.mean()

print("\nRandom Forest Mean Validation Metrics:")
print(mean_rf_results)

print("\nSVM Mean Validation Metrics:")
print(mean_svm_results)

print("\nGradient boosing Mean Validation Metrics:")
print(mean_gb_results)


Neural Network Mean Validation Metrics:
   units1  units2  dropout_rate  batch_size  epochs     lr  val_loss  \
1      32      16           0.2          32      50  0.010  0.646355   
5      64      16           0.2          32      50  0.010  0.647074   
6      64      32           0.2          32      50  0.001  0.644838   
7      64      32           0.2          32      50  0.010  0.645760   
0      32      16           0.2          32      50  0.001  0.646922   
2      32      32           0.2          32      50  0.001  0.649412   
3      32      32           0.2          32      50  0.010  0.644242   
4      64      16           0.2          32      50  0.001  0.640641   

   val_accuracy  f1_score       auc       tpr       tnr       fpr       fnr  
1      0.640602  0.625436  0.679094  0.605479  0.675889  0.324111  0.394521  
5      0.636090  0.640879  0.682611  0.653927  0.621041  0.378959  0.346073  
6      0.631579  0.626924  0.681231  0.622149  0.641780  0.358220  0.377851 

In [260]:
forpred = pd.read_parquet('test_with_pairs_Clean.parquet')

In [261]:
pairs = forpred['pair']

In [262]:
feature_columns = data.columns.drop(['Unnamed: 0', 'target', 'CADD_METSIM'])

In [263]:
## Train on the full
data = pd.read_csv('20240923_TrainingDataSet_Clean.csv')
data['target'] = data['target'].replace(-1, 0)
data = data.replace([np.inf, -np.inf], np.nan)
data = data.dropna()
x_all = data[feature_columns].values
y_all = data['target'].values

In [264]:
# Train final Random Forest model on all data
final_rf_model = RandomForestClassifier(random_state=123, min_samples_split=5, max_depth=10, n_estimators=200)
final_rf_model.fit(x_all, y_all)

# Evaluate final Random Forest model
rf_pred_all = final_rf_model.predict(x_all)
rf_pred_prob_all = final_rf_model.predict_proba(x_all)[:, 1]

rf_accuracy_all = accuracy_score(y_all, rf_pred_all)
rf_f1_all = f1_score(y_all, rf_pred_all)
rf_auc_all = roc_auc_score(y_all, rf_pred_prob_all)
tn, fp, fn, tp = confusion_matrix(y_all, rf_pred_all).ravel()
rf_tpr = tp / (tp + fn)
rf_tnr = tn / (tn + fp)
rf_fpr = fp / (fp + tn)
rf_fnr = fn / (fn + tp)

In [265]:
# importance
importance_df = pd.DataFrame({
    'Feature': feature_columns,
    'Importance': final_rf_model.feature_importances_
})

# Sort the DataFrame by importance
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Display the feature importance DataFrame
print(importance_df)

# Output as dataframe 
importance_df.to_csv('importance_results.csv', index=False)  # Set index=False to avoid writing row numbers

                    Feature  Importance
11   Neighb_Rxn_MinP_METSIM    0.090316
12       Neighb_TWAS_METSIM    0.089406
5         Neighb_Rxn_N_CLSA    0.078090
9           Min pval_METSIM    0.072111
3      Neighb_Rxn_MinP_CLSA    0.069906
4          Neighb_TWAS_CLSA    0.068230
7     Neighb_Chem_TWAS_CLSA    0.061834
2                 CADD_CLSA    0.060054
13      Neighb_Rxn_N_METSIM    0.058214
1                 TWAS_CLSA    0.057464
0             Min pval_CLSA    0.050951
6     Neighb_Chem_MinP_CLSA    0.050460
10              TWAS_METSIM    0.048501
15  Neighb_Chem_TWAS_METSIM    0.046055
14  Neighb_Chem_MinP_METSIM    0.042857
8        Neighb_Chem_N_CLSA    0.028620
16     Neighb_Chem_N_METSIM    0.026929


In [266]:
forpred = forpred[feature_columns]
rf_pred = final_rf_model.predict_proba(forpred.values)[:,1]

In [267]:
y_pred_custom_threshold = (rf_pred >= 0.697775).astype(int)

In [268]:
# how many positive associations there are
sum(y_pred_custom_threshold)

123930

In [269]:
output_prediction = pd.DataFrame({
    'Pairs': pairs,
    'Prob': rf_pred,
    'Class': y_pred_custom_threshold
})

In [270]:
len(rf_pred)

2426064

In [271]:
output_prediction.to_csv('20240924_predictions.csv', index=False)